# Exercise 2

https://www.sciencedirect.com/science/article/abs/pii/S0377221705005643

In [1]:
import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
n = 2 # number of servers in the simulation
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 10100 # Simulation length in minutes
rho = 0.95
mu = .75
SERVICETIME = 1/mu # average minutes to complete a task

wtime = 100
btime = 100
mtime = 400

# def monitor(data, resource):
#     """This is a monitoring callback"""
#     item = [
#         resource._env.now,
#         resource.count,
#         len(resource.queue)
#     ]
#     data.append(item)

class Server(object):
    
    def __init__(self, env, num_server, servicetime):
        
        self.env = env
        self.server = sp.Resource(env, num_server)
        self.servicetime = servicetime
        
    def service(self, customer):
        # need to add stochasticity to this
        
        yield self.env.timeout(randomSample(SERVICETIME))
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, data):
    
    arrival = env.now
    #print("%s arrives at %.2f." %(name, arrival))
    
    if env.now%500 > 100:
        data[0].append(len(sv.server.queue))
    
    with sv.server.request() as request:
        
        startwaiting = env.now
        yield request
        waitingtime = env.now-startwaiting
        
        service = env.now
#         print("%s is processed at %.2f." % (name, service))
        
        if env.now%500 > 100:
            data[0].append(len(sv.server.queue))
            data[1].append(waitingtime)
        
        yield env.process(sv.service(name))
        
#         print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, data):
    
    server = Server(env, num_servers, servicetime)
    
#     # create 4 customers
#     for i in range(4*num_servers):
#         data[0] += 1
#         env.process(client(env, 'Client %d' % data[0], server, data))


    # make more customers while the simulation is running
    i = 0
    while True:
        i+=1
        yield env.timeout(randomSample(1/lambd))
        
        env.process(client(env, 'Client %d' % i, server, data))

        
def runSimulation(serverNumber, data):
    
    #random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, data))

    env.run(SIM_TIME)

#     print("The total waiting time was %.2f" %(data[2]))
#     print("There were %i clients, %i of who(m?) had to wait." %(data[0],data[1]))
#     print("The average waiting time was %.2f" %(data[2]/data[0]))


In [2]:
import csv

def saveData(results):
    
    filename = 'resultsqueue.csv'
        
    with open(filename, 'a', newline = '') as csvfile:
        writer = csv.writer(csvfile, delimiter=',', quotechar='"')
        
        writer.writerow(results)
            
    
def calcurmv(xj, sj, xc, j):
    xj1 = xj + ((xc-xj)/(j+1))
    sj1 = (((1-(1/j))*(sj**2)) + (j+1)*((xj1-xj)**2))**0.5
    return xj1, sj1

def checkstop(s,k,l):
    
    if (2*za*s/(k)**0.5)<l:
        print(2*za*s/(k)**0.5)
        return True
    else:
        return False

In [3]:
data = [[],[]]

timedata = []
quedata = []

nlist = [1,2,4]

za = 2.975

for n in nlist:
    for rho in np.arange(0.1, 1, 0.1):
        
        print(n,rho)
        lambd = rho * mu * n

        runSimulation(n, data)

        quelen = np.array(data[0])
        stime = np.array(data[1])

        xqm = np.mean(quelen)
        xsm = np.mean(stime)

        sq = np.var(quelen)
        ss = np.var(stime)
        
        sqstop = True
        ssstop = True

        results = [0 for j in range(10)]
        results[0] = n
        results[1] = rho
         
        i = 1

        while True:
            print("Running..", i, sq, ss)

            data = [[],[]]
            runSimulation(n, data)

            quelen = np.array(data[0])
            stime = np.array(data[1])
            xqt = np.mean(quelen)
            xst = np.mean(stime)

            xqm, sq = calcurmv(xqm, sq, xqt, i)
            xsm, ss = calcurmv(xsm, ss, xst, i)

            if i>=100:
                print("Checkstop can activate")
                if sqstop and checkstop(sq, i, xqm / 10):
                    
                    results[2] = xqm
                    results[3] = sq
                    results[4] = i
                    results[5] = 2*za*sq/(i)**0.5
                    sqstop = False
                
                if ssstop and checkstop(ss, i, xsm / 10):
                    
                    results[6] = xsm
                    results[7] = ss
                    results[8] = i
                    results[9] = 2*za*ss/(i)**0.5
                    ssstop = False
                
                if not sqstop and not ssstop:
                    break
            i+=1
        
        saveData(results)


1 0.1
Running.. 1 0.006638707601641253 0.26698126393698846
Running.. 2 1.965831887501649e-05 0.006528912657135061
Running.. 3 0.0017457193415921818 0.011905883497219272
Running.. 4 0.001897458084463721 0.009912941115976995
Running.. 5 0.0018737535657424775 0.012131990450359675
Running.. 6 0.0026654271115997613 0.01190310184137214
Running.. 7 0.003616901011653815 0.013197885577566676
Running.. 8 0.004093239867163418 0.014285980268444704
Running.. 9 0.004198295194963019 0.027682813859198765
Running.. 10 0.007220819159089489 0.03435165409478132
Running.. 11 0.006862533533304494 0.03269742014874549
Running.. 12 0.007140059321007138 0.035871284479732486
Running.. 13 0.006900700469940142 0.03448932102358936
Running.. 14 0.006710606251202327 0.03344679640490973
Running.. 15 0.006507130230858636 0.03242933303065332
Running.. 16 0.006445614611788604 0.03133534158076779
Running.. 17 0.006476130990537313 0.031102707035498972
Running.. 18 0.006343823637372083 0.030574015347171638
Running.. 19 0.00

Running.. 52 0.012346554309427849 0.04049998282477854
Running.. 53 0.012305618512861877 0.040409655546485374
Running.. 54 0.012352472734720617 0.04091361676781077
Running.. 55 0.01238910366356819 0.04067993080620131
Running.. 56 0.012299102478897771 0.04053796589779239
Running.. 57 0.012188807555970608 0.04096683503841356
Running.. 58 0.012262063190884647 0.04070071270183674
Running.. 59 0.012484468598398236 0.041602172199423464
Running.. 60 0.012395585895851569 0.04148897642074119
Running.. 61 0.012881583171168515 0.042778665052034114
Running.. 62 0.012776653763756871 0.04242809508858552
Running.. 63 0.012745233328959757 0.0429013837248357
Running.. 64 0.012656117854681521 0.04271760110632724
Running.. 65 0.012721420872006412 0.04270684836695349
Running.. 66 0.0126234737316352 0.04247996150298934
Running.. 67 0.012528345592338381 0.04217929563999493
Running.. 68 0.012597606387214478 0.04187036729271374
Running.. 69 0.012785468502813004 0.041858987600606176
Running.. 70 0.0127140964516

Running.. 3 0.06846758811238612 0.16181972235501468
Running.. 4 0.06064533332852205 0.14078622248601425
Running.. 5 0.05512806398605904 0.1262267705947361
Running.. 6 0.05538298091863702 0.11860710316467224
Running.. 7 0.05094661619663549 0.10875298462639911
Running.. 8 0.051888159255882464 0.10781724186283895
Running.. 9 0.04938181018681835 0.10344747500979763
Running.. 10 0.0465668183536062 0.09753289560602887
Running.. 11 0.045026690629413786 0.09556236154710056
Running.. 12 0.0432426279511439 0.09508098076532592
Running.. 13 0.044033985949477175 0.0918354361142018
Running.. 14 0.04251593420180568 0.08843665084489795
Running.. 15 0.041111367752162085 0.08535364234658109
Running.. 16 0.040822200245988136 0.08630045230836826
Running.. 17 0.041485598854513424 0.08922958454862008
Running.. 18 0.04215979891466517 0.09156412929065687
Running.. 19 0.04132557206580719 0.09007061510024617
Running.. 20 0.040347494278898874 0.088531539634873
Running.. 21 0.03999829735117104 0.08848136663535289

Running.. 57 0.0631920932693734 0.12731066183414444
Running.. 58 0.06289002491079437 0.1265506548128931
Running.. 59 0.06330991864192756 0.12778161966091633
Running.. 60 0.06299202130674426 0.12697222459507532
Running.. 61 0.06297307165191732 0.12749126757515444
Running.. 62 0.06254042767599402 0.1265589304313894
Running.. 63 0.06235303683605044 0.1260915512285214
Running.. 64 0.06185677474458363 0.12512207056379562
Running.. 65 0.06147002234076641 0.12427730447300336
Running.. 66 0.061061319333866014 0.1235236767633149
Running.. 67 0.06063762787245691 0.12285033246428492
Running.. 68 0.06063611354914596 0.12372295684452432
Running.. 69 0.060200425406998125 0.12287257418953168
Running.. 70 0.060826421231825016 0.12385653105101826
Running.. 71 0.06055317655285126 0.12415008703449947
Running.. 72 0.061529836971697276 0.12714610611775948
Running.. 73 0.0627005874200346 0.12854069137820595
Running.. 74 0.0622782110548874 0.12769031364641678
Running.. 75 0.06188575916588435 0.12682775454442

Running.. 11 0.23822147094013219 0.38066920671357374
Running.. 12 0.23312732868008051 0.38275164345855517
Running.. 13 0.2238126690661073 0.36663149982893645
Running.. 14 0.21534799989602277 0.35718499124123976
Running.. 15 0.23350519183946525 0.37348756572562775
Running.. 16 0.22578259091287262 0.3623626611105678
Running.. 17 0.22113102968957982 0.3581650991876252
Running.. 18 0.21766276531990375 0.3480378927728116
Running.. 19 0.21199869550840963 0.3384578388854012
Running.. 20 0.21252960926360473 0.3384404893979709
Running.. 21 0.20750569717340686 0.33000785932802734
Running.. 22 0.2306822664628674 0.3569089581795869
Running.. 23 0.22809891499678742 0.3552877447173214
Running.. 24 0.22439354901081032 0.3496697715069656
Running.. 25 0.22062864360754456 0.34442322157093075
Running.. 26 0.21705884395759806 0.338713958987077
Running.. 27 0.2128909051872909 0.3323935604392165
Running.. 28 0.21079579137355095 0.33024054506200934
Running.. 29 0.2072102728252277 0.3245304100026533
Running..

...

Running.. 89 0.004346008789088681 0.0032570669484207134
Running.. 90 0.004324544507401283 0.003250638898131931
Running.. 91 0.004346219488922128 0.0032550263540336694
Running.. 92 0.004332049676467497 0.0032396284622237754
Running.. 93 0.0043241532665049 0.0032362829492716323
Running.. 94 0.00430086300658106 0.0032217377604975292
Running.. 95 0.004323564035597437 0.003255058235433525
Running.. 96 0.004303620163171079 0.0032388280332919465
Running.. 97 0.004304488618945782 0.003225729836722002
Running.. 98 0.004282855339776629 0.0032108971230674225
Running.. 99 0.004262204318521789 0.003195453960707737
Running.. 100 0.00424903873083247 0.0031828617738483547
Checkstop can activate
0.0025201547122491673
0.0018855196954544694
4 0.4
Running.. 1 0.09048863723434988 0.0377139315353478
Running.. 2 0.022601472154592014 0.01573821223168839
Running.. 3 0.017773854448352035 0.013127265038956752
Running.. 4 0.01452379136445176 0.010723976117513353
Running.. 5 0.012736919547772669 0.0093331729344591

Running.. 37 0.01964178856872731 0.010836943233676954
Running.. 38 0.01941137257292619 0.010722760389966569
Running.. 39 0.02020554957187717 0.011259051903529396
Running.. 40 0.020205458362171906 0.011291021831791087
Running.. 41 0.020401323037406112 0.011449686982376998
Running.. 42 0.020173508047876064 0.011316398833655782
Running.. 43 0.019953915949465495 0.011207244620575077
Running.. 44 0.02037860702494972 0.011224241395964204
Running.. 45 0.020678813951781585 0.01156009421493271
Running.. 46 0.0204787059231974 0.011447753364596639
Running.. 47 0.020258415392327 0.01132282591768568
Running.. 48 0.02011389908086563 0.011253201427320381
Running.. 49 0.019912901633256416 0.0111385571372954
Running.. 50 0.020233851145602413 0.011355164270557919
Running.. 51 0.020124904888378568 0.011244137639065458
Running.. 52 0.01998566026597116 0.011228474855245252
Running.. 53 0.0199720179316417 0.011186375000585159
Running.. 54 0.020266499323908723 0.011326395124417738
Running.. 55 0.020081135215

Running.. 88 0.05210506494321763 0.023172730503838983
Running.. 89 0.051915869095430195 0.023069383072130526
Running.. 90 0.05164572199565306 0.02294165285111395
Running.. 91 0.05177698400263533 0.022932282891777605
Running.. 92 0.05149506206492907 0.02281041600115804
Running.. 93 0.05166608167179999 0.022898673241099225
Running.. 94 0.05207942121923511 0.023075620321685475
Running.. 95 0.05208498052679352 0.023062688539154313
Running.. 96 0.05181112508121318 0.022941834791407045
Running.. 97 0.051654416892386104 0.022860336755125728
Running.. 98 0.05161670451305653 0.022874181890805662
Running.. 99 0.05145024483206962 0.02280007594547246
Running.. 100 0.051372137430108655 0.02280816773742887
Checkstop can activate
0.030509968909124312
0.013560856468139195
4 0.7000000000000001
Running.. 1 4.357478398217649 0.7792436188314051
Running.. 2 0.1783656805417099 0.07118932340828434
Running.. 3 0.12761081701409566 0.050780844417506604
Running.. 4 0.10539531826906527 0.04259491000422722
Running

Running.. 41 0.3333671055277332 0.12746814643406792
Running.. 42 0.33561648538966293 0.1285275000830046
Running.. 43 0.3316489976534726 0.12698855448603116
Running.. 44 0.32862279177513165 0.1258019435778178
Running.. 45 0.3249148043211168 0.12437102849654401
Running.. 46 0.3214265392497415 0.12305606241828719
Running.. 47 0.32027252007605733 0.12265189621190925
Running.. 48 0.3179518815032668 0.12180863880616148
Running.. 49 0.31557208613865034 0.12098541819997956
Running.. 50 0.31533877138531713 0.12065806355366425
Running.. 51 0.31326018387353133 0.11980507426171567
Running.. 52 0.3104392292380444 0.1187250212233863
Running.. 53 0.3089979923994845 0.1181099013711006
Running.. 54 0.3084987624998577 0.11794152649117035
Running.. 55 0.3067522404630668 0.11725201624384884
Running.. 56 0.3073272461513066 0.11730494308898802
Running.. 57 0.3047857151241848 0.1163255705265811
Running.. 58 0.30611818559660475 0.11698156995371158
Running.. 59 0.3041558816541985 0.11632445250041068
Running.. 

Running.. 99 1.0550100334071824 0.373761697955401
Running.. 100 1.0537407479334693 0.3730388680852329
Checkstop can activate
0.6241357589255389
0.2209785139128782


the link https://simpy.readthedocs.io/en/latest/examples/carwash.html

In [93]:
data = [0,0,0]

n = 1
lambd = rho * mu * n

runSimulation(n, data)


Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 1 is processed at 0.00.
Client 1 is finished at 0.77.
Client 2 is processed at 0.77.
Client 5 arrives at 0.79.
Client 6 arrives at 0.83.
Client 2 is finished at 1.76.
Client 3 is processed at 1.76.
Client 7 arrives at 1.79.
Client 8 arrives at 2.40.
Client 9 arrives at 2.90.
Client 10 arrives at 3.45.
Client 11 arrives at 5.14.
Client 3 is finished at 5.20.
Client 4 is processed at 5.20.
Client 4 is finished at 5.75.
Client 5 is processed at 5.75.
Client 12 arrives at 6.43.
Client 13 arrives at 6.63.
Client 14 arrives at 6.94.
Client 15 arrives at 7.83.
Client 16 arrives at 8.05.
Client 5 is finished at 8.27.
Client 6 is processed at 8.27.
Client 6 is finished at 8.84.
Client 7 is processed at 8.84.
Client 17 arrives at 9.22.
Client 18 arrives at 9.54.
Client 19 arrives at 9.56.
Client 20 arrives at 10.14.
Client 7 is finished at 10.74.
Client 8 is processed at 10

In [98]:
data = [0,0,0]

n = 4
lambd = rho * mu * n

runSimulation(n, data)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 5 arrives at 0.00.
Client 6 arrives at 0.00.
Client 7 arrives at 0.00.
Client 8 arrives at 0.00.
Client 9 arrives at 0.00.
Client 10 arrives at 0.00.
Client 11 arrives at 0.00.
Client 12 arrives at 0.00.
Client 13 arrives at 0.00.
Client 14 arrives at 0.00.
Client 15 arrives at 0.00.
Client 16 arrives at 0.00.
Client 1 is processed at 0.00.
Client 2 is processed at 0.00.
Client 3 is processed at 0.00.
Client 4 is processed at 0.00.
Client 2 is finished at 0.18.
Client 5 is processed at 0.18.
Client 17 arrives at 0.38.
Client 5 is finished at 0.48.
Client 6 is processed at 0.48.
Client 18 arrives at 0.61.
Client 3 is finished at 0.73.
Client 7 is processed at 0.73.
Client 19 arrives at 0.86.
Client 20 arrives at 0.89.
Client 21 arrives at 1.02.
Client 22 arrives at 1.17.
Client 4 is finished at 1.38.
Client 8 is processed at 1.38.
Client 23 arrives at 1.75.
Client 

# Exercise 3

In [99]:
import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
n = 2 # number of servers in the simulation
SERVICETIME = 1.5 # average? minutes to complete a task
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 50 # Simulation length in minutes
rho = 0.5
mu = 2

class Server(object):
    
    def __init__(self, env, num_server):
        
        self.env = env
        self.server = sp.PriorityResource(env, num_server)
        
    def service(self,serviceTime):
        # need to add stochasticity to this
        
        yield self.env.timeout(serviceTime)
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, waiting):
    
    arrival = env.now
    print("%s arrives at %.2f." %(name, arrival))
    
    serviceTime = randomSample(SERVICETIME)
    
    with sv.server.request(priority=serviceTime) as request:
        
        yield request
        
        service = env.now
        print("%s is processed at %.2f." % (name, service))
        
        if arrival != service:
            waiting[1] += 1
            waiting[2] += service - arrival
        
        yield env.process(sv.service(serviceTime))
        
        print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, waiting):
    
    server = Server(env, num_servers)
    
    # create 4 customers
    for i in range(4*num_servers):
        waiting[0] += 1
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        
    # make more customers while the simulation is running
    while waiting[0]<40*num_servers:
        yield env.timeout(randomSample(1/lambd))
        waiting[0] += 1 
        
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        

def runSimulation(serverNumber, waiting):
    
    print('Server is up')
    random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, waiting))

    env.run()

    print("The total waiting time was %.2f" %(waiting[2]))
    print("There were %i clients, %i of who(m?) had to wait." %(waiting[0],waiting[1]))
    print("The average waiting time was %.2f" %(waiting[2]/waiting[0]))
    
    

In [100]:
waiting = [0,0,0]

n = 1
lambd = rho * mu * n

runSimulation(n, waiting)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 1 is processed at 0.00.
Client 1 is finished at 1.23.
Client 4 is processed at 1.23.
Client 4 is finished at 1.31.
Client 2 is processed at 1.31.
Client 2 is finished at 2.21.
Client 3 is processed at 2.21.
Client 5 arrives at 2.22.
Client 6 arrives at 3.51.
Client 7 arrives at 4.24.
Client 3 is finished at 4.34.
Client 6 is processed at 4.34.
Client 6 is finished at 4.80.
Client 5 is processed at 4.80.
Client 8 arrives at 4.81.
Client 5 is finished at 5.51.
Client 8 is processed at 5.51.
Client 8 is finished at 6.71.
Client 7 is processed at 6.71.
Client 9 arrives at 6.91.
Client 10 arrives at 8.19.
Client 7 is finished at 9.19.
Client 10 is processed at 9.19.
Client 10 is finished at 9.59.
Client 9 is processed at 9.59.
Client 11 arrives at 11.20.
Client 12 arrives at 12.34.
Client 13 arrives at 12.75.
Client 14 arrives at 13.12.
Client 15 arrives at 13.98.
Clie

In [98]:
waiting = [0,0,0]

n = 2
lambd = rho * mu * n

runSimulation(n, waiting)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 5 arrives at 0.00.
Client 6 arrives at 0.00.
Client 7 arrives at 0.00.
Client 8 arrives at 0.00.
Client 9 arrives at 0.00.
Client 10 arrives at 0.00.
Client 11 arrives at 0.00.
Client 12 arrives at 0.00.
Client 13 arrives at 0.00.
Client 14 arrives at 0.00.
Client 15 arrives at 0.00.
Client 16 arrives at 0.00.
Client 1 is processed at 0.00.
Client 2 is processed at 0.00.
Client 3 is processed at 0.00.
Client 4 is processed at 0.00.
Client 2 is finished at 0.18.
Client 5 is processed at 0.18.
Client 17 arrives at 0.38.
Client 5 is finished at 0.48.
Client 6 is processed at 0.48.
Client 18 arrives at 0.61.
Client 3 is finished at 0.73.
Client 7 is processed at 0.73.
Client 19 arrives at 0.86.
Client 20 arrives at 0.89.
Client 21 arrives at 1.02.
Client 22 arrives at 1.17.
Client 4 is finished at 1.38.
Client 8 is processed at 1.38.
Client 23 arrives at 1.75.
Client 

In [98]:
waiting = [0,0,0]

n = 4
lambd = rho * mu * n

runSimulation(n, waiting)

Server is up
Client 1 arrives at 0.00.
Client 2 arrives at 0.00.
Client 3 arrives at 0.00.
Client 4 arrives at 0.00.
Client 5 arrives at 0.00.
Client 6 arrives at 0.00.
Client 7 arrives at 0.00.
Client 8 arrives at 0.00.
Client 9 arrives at 0.00.
Client 10 arrives at 0.00.
Client 11 arrives at 0.00.
Client 12 arrives at 0.00.
Client 13 arrives at 0.00.
Client 14 arrives at 0.00.
Client 15 arrives at 0.00.
Client 16 arrives at 0.00.
Client 1 is processed at 0.00.
Client 2 is processed at 0.00.
Client 3 is processed at 0.00.
Client 4 is processed at 0.00.
Client 2 is finished at 0.18.
Client 5 is processed at 0.18.
Client 17 arrives at 0.38.
Client 5 is finished at 0.48.
Client 6 is processed at 0.48.
Client 18 arrives at 0.61.
Client 3 is finished at 0.73.
Client 7 is processed at 0.73.
Client 19 arrives at 0.86.
Client 20 arrives at 0.89.
Client 21 arrives at 1.02.
Client 22 arrives at 1.17.
Client 4 is finished at 1.38.
Client 8 is processed at 1.38.
Client 23 arrives at 1.75.
Client 

# Exercise 4

In [ ]:
# M/D/n queue

import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
n = 2 # number of servers in the simulation
SERVICETIME = 1.5 # average? minutes to complete a task
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 50 # Simulation length in minutes
rho = 0.5
mu = 1/SERVICETIME

class Server(object):
    
    def __init__(self, env, num_server, servicetime):
        
        self.env = env
        self.server = sp.Resource(env, num_server)
        self.servicetime = servicetime
        
    def service(self, customer):
        # need to add stochasticity to this
        
        yield self.env.timeout(SERVICETIME)
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, waiting):
    
    arrival = env.now
    print("%s arrives at %.2f." %(name, arrival))
    
    with sv.server.request() as request:
        
        yield request
        
        service = env.now
        print("%s is processed at %.2f." % (name, service))
        
        if arrival != service:
            waiting[1] += 1
            waiting[2] += service - arrival
        
        yield env.process(sv.service(name))
        
        print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, waiting):
    
    server = Server(env, num_servers, servicetime)
    
    # create 4 customers
    for i in range(4*num_servers):
        waiting[0] += 1
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        
    # make more customers while the simulation is running
    while waiting[0]<40*num_servers:
        yield env.timeout(randomSample(1/lambd))
        waiting[0] += 1 
        
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        

def runSimulation(serverNumber, waiting):
    
    print('Server is up')
    random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, waiting))

    env.run()

    print("The total waiting time was %.2f" %(waiting[2]))
    print("There were %i clients, %i of who(m?) had to wait." %(waiting[0],waiting[1]))
    print("The average waiting time was %.2f" %(waiting[2]/waiting[0]))

In [ ]:
# longtail distribution

import simpy as sp
import numpy as np
import pandas as pd
import seaborn as sns
import random

RANDOM_SEED = 42
n = 2 # number of servers in the simulation
SERVICETIMELARGE = 5.0 # average? minutes to complete a task
SERVICETIMESMALL = 1.0
T_ARRIVAL = 7 # arrival of a new customer
SIM_TIME = 50 # Simulation length in minutes
rho = 0.5
mu = 0.5

class Server(object):
    
    def __init__(self, env, num_server, servicetime):
        
        self.env = env
        self.server = sp.Resource(env, num_server)
        self.servicetime = servicetime
        
    def service(self, customer):
        # need to add stochasticity to this
        
        if np.random.randint(0,3) == 3:
            yield self.env.timeout(randomSample(SERVICETIMELARGE))
            
        else:
            yield self.env.timeout(randomSample(SERVICETIMESMALL))
        
def randomSample(rate):
    
    return np.random.exponential(rate)
        
def client(env, name, sv, waiting):
    
    arrival = env.now
    print("%s arrives at %.2f." %(name, arrival))
    
    with sv.server.request() as request:
        
        yield request
        
        service = env.now
        print("%s is processed at %.2f." % (name, service))
        
        if arrival != service:
            waiting[1] += 1
            waiting[2] += service - arrival
        
        yield env.process(sv.service(name))
        
        print("%s is finished at %.2f." %(name, env.now))
        
        
def setup(env, num_servers, servicetime, t_arrival, waiting):
    
    server = Server(env, num_servers, servicetime)
    
    # create 4 customers
    for i in range(4*num_servers):
        waiting[0] += 1
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        
    # make more customers while the simulation is running
    while waiting[0]<40*num_servers:
        
        yield env.timeout(randomSample(1/lambd))
        waiting[0] += 1 
        
        env.process(client(env, 'Client %d' % waiting[0], server, waiting))
        

def runSimulation(serverNumber, waiting):
    
    print('Server is up')
    random.seed(RANDOM_SEED)

    # create the enviroment and start the setup process

    env = sp.Environment()
    env.process(setup(env, serverNumber, SERVICETIME, T_ARRIVAL, waiting))

    env.run()

    print("The total waiting time was %.2f" %(waiting[2]))
    print("There were %i clients, %i of who(m?) had to wait." %(waiting[0],waiting[1]))
    print("The average waiting time was %.2f" %(waiting[2]/waiting[0]))